In [1]:
from openai import OpenAI
import numpy as np

In [7]:
client = OpenAI(api_key="sk-svcacct-oD2ywALJLsVNQwU3GZ-Lvl-6d53TV1hwjr7DKEfvT07SSYDzFHvmTo3Fx9gp-5KT3BlbkFJsc-MOn7QvehX14nXoONvR0tKAQ8_XBfDFywvS3CVLyL4QaCm9Pklco6v0i3hhAA")

In [3]:
with open('messi.txt', 'r') as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [10]:
CHUNNK_SIZE = 50 #every 100 letters will be put into chunks list
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['Lionel Andrés Messi[note 1] (Spanish pronunciation', ': [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', 'lso known as Leo Messi, is an Argentine profession', 'al footballer who plays as a forward for and capta', 'ins both Major League Soccer club Inter Miami and ', 'the Argentina national team. Widely regarded as on', 'e of the greatest players of all time, Messi set n', 'umerous records for individual accolades won throu', 'ghout his professional footballing career such as ', "eight Ballon d'Or awards and eight times being nam", "ed the world's best player by FIFA.[note 2] He is ", 'the most decorated player in the history of profes', 'sional football having won 45 team trophies,[note ', '3] including twelve league titles, four UEFA Champ', 'ions Leagues, two Copa Américas, and one FIFA Worl', 'd Cup. Messi holds the records for most European G', 'olden Shoes (6), most goals for a single club (672', ', with Barcelona), most goals (474), hat-tricks (3', '6) and assists (192) in La

In [11]:
embedded_chunks = {} # -> [(text, embedding), (text, embedding), (...,...), (...,...)]
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks[chunk] = response.data[0].embedding
print(embedded_chunks)

{'Lionel Andrés Messi[note 1] (Spanish pronunciation': [0.03845192492008209, -0.06453130394220352, -0.041848309338092804, -0.002546024974435568, -0.03881582245230675, 0.01881152205169201, 0.012028861790895462, 0.01082597579807043, -0.03558117151260376, -0.016173260286450386, -0.07136450707912445, 0.009284461848437786, -0.010153775103390217, -0.03214435279369354, 0.027797790244221687, 0.0530887246131897, -0.05895153060555458, -0.01287795789539814, 0.00017026146815624088, -0.04957104101777077, 0.030102480202913284, -0.012332110665738583, 0.016436075791716576, 0.01854870654642582, 0.025371801108121872, 0.0034570342395454645, -0.051147934049367905, 0.011806479655206203, -0.0009198541520163417, 0.015677953138947487, 0.07658038288354874, -0.02450248785316944, 0.010431752540171146, 0.0002070619520964101, -0.012867850251495838, -0.017345821484923363, -0.029718365520238876, -0.01958986185491085, -0.00481912586838007, -0.044638197869062424, -0.018366757780313492, -0.03790607675909996, 0.01888228

In [29]:
def cosine_similarity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )
    #-> ABCos/ (A/||A|| * B/||B||)

In [16]:
question = "what team does Messi play for?"

In [17]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[-0.016306936740875244, -0.06608225405216217, -0.015961062163114548, 0.0038325877394527197, -0.04398702085018158, 0.01793457753956318, -0.00040118827018886805, 0.0315965972840786, -0.012349732220172882, -0.019348591566085815, -0.08471875637769699, 0.03586915507912636, -0.038269929587841034, 0.0015678767813369632, 0.016093308106064796, 0.05017205700278282, -0.03253249078989029, -0.0315559059381485, -0.03772059828042984, 0.013509427197277546, -0.018687361851334572, -0.004432780668139458, -0.019724983721971512, 0.010208365507423878, 0.007441374938935041, 0.01004051510244608, -0.021261071786284447, 0.03871753066778183, -0.008026308380067348, 0.05619433522224426, 0.05631640553474426, -0.04809681698679924, 0.05151486396789551, 0.016490045934915543, 0.0043564848601818085, -0.022176619619131088, -0.017476804554462433, -0.016052616760134697, 0.02526913955807686, -0.0037893536500632763, 0.011210382916033268, -0.035197753459215164, -0.005180478561669588, 0.028524423018097878, -0.01802613213658332

In [30]:
similarities = []
keys = list(embedded_chunks.keys())
for key in keys:
    similarity = cosine_similarity(question_embedding, embedded_chunks[key])
    similarities.append((key, similarity))
top_5_chunks = sorted(similarities, key=lambda x: x[1], reverse=True)[:5]

In [31]:
combined_info = " ".join([chunk for chunk, _ in top_5_chunks])

In [32]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": f"User question: {question}. Potentially useful information: {combined_info}"
    }
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

In [33]:
print(completion.choices[0].message.content)

As of 2023, Lionel Messi plays for Inter Miami CF in Major League Soccer (MLS). Before joining Inter Miami, he played for Paris Saint-Germain (PSG) and is also known for his long tenure at FC Barcelona. Additionally, Messi is a prominent player for the Argentina national team.
